## Audio Weather Classification Script

This script is designed to perform audio classification using convolutional neural networks (CNNs). It loads audio files from the specified directory, preprocesses them by converting them to mel spectrograms, pads them to ensure uniform dimensions, and then trains a CNN model for classifying the audio into different classes.

### Steps:
1. **Data Loading and Preprocessing**: Audio files are loaded from the specified directory (`path`) and converted to mel spectrograms using the Librosa library. The spectrograms are then padded to ensure uniform dimensions.

2. **Data Splitting**: The preprocessed data is split into training and testing sets using a specified test size.

3. **Model Architecture**: The CNN model architecture consists of two convolutional layers followed by max-pooling layers and dropout regularization. The output is passed through fully connected layers with ReLU activation functions, and a softmax activation function is used for multi-class classification.

4. **Training**: The model is trained using the training data, and the training progress is monitored using accuracy and loss metrics.

### Libraries Used:
- Librosa: For audio processing
- NumPy: For numerical operations
- TensorFlow and Keras: For building and training the CNN model
- Scikit-learn: For data preprocessing and splitting

Feel free to modify the script and experiment with different parameters to improve model performance.

### Import Libraries

In [2]:
import os
import librosa
#import librosa.display
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### Data Preprocessing

In [3]:
def load_and_pad_audio(file_path, target_length=216):
    # Load audio file
    audio, sr = librosa.load(file_path, sr=None)
    # Calculate the number of required samples
    required_samples = int(target_length * sr / 10.0)
    if len(audio) < required_samples:
        # Pad audio if it is too short
        audio = np.pad(audio, (0, max(0, required_samples - len(audio))), "constant")
    else:
        # Truncate audio if it is too long
        audio = audio[:required_samples]
    return audio, sr

def load_audio_files(path, class_labels):
    audios = []
    labels = []
    for label in class_labels:
        # Path to the folder containing audio files of a specific label
        class_path = os.path.join(path, label)
      
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            audio, sr = load_and_pad_audio(file_path)
            # Convert to mel spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
            log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
            audios.append(log_mel_spectrogram)
            labels.append(label)
        
    return audios, labels

class_labels = ["Rain", "Thunder"]  # Class names ("Rain", "Thunder", "Earthquake", "Flood" , "Tornado", "Volcano")
path = r'D:\Weather_Sounds\Weather_sounds'  # Modify this to your path
audios, labels = load_audio_files(path, class_labels)

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Finding max dimensions for padding
max_length = max(audio.shape[1] for audio in audios)
max_height = max(audio.shape[0] for audio in audios)

# Pad spectrograms to the max dimensions
audios_padded = np.array([np.pad(audio, ((0, max_height - audio.shape[0]), (0, max_length - audio.shape[1])), 'constant') for audio in audios])

# Reshape for CNN input
audios_array = audios_padded.reshape(audios_padded.shape[0], audios_padded.shape[1], audios_padded.shape[2], 1)

print(audios_array.shape)
print(labels_categorical.shape)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

TypeError: cannot unpack non-iterable NoneType object

In [4]:
print(audios_array.shape)
print(labels_categorical.shape)
print(len(labels))

(7895, 128, 1861, 1)
(7895, 2)
7895


In [ ]:

# Split data
X_train, X_test, y_train, y_test = train_test_split(audios_array, labels_categorical, test_size=0.2, random_state=42)

### Building Model

In [5]:
def build_model(input_shape, num_classes):
    model = Sequential()
    
    # First conv block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Second conv block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Flatten and Dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(input_shape=(X_train.shape[1], X_train.shape[2], 1), num_classes=len(class_labels))

### Training Model

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=1)

### Analysis

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

#### Saving Model

In [ ]:
# Define the path to save the model PB format
model_dir = "weather_audio_detection_model"

# Save the model in SavedModel format
tf.saved_model.save(model, model_dir)

INFO:tensorflow:Assets written to: weather_audio_detection_model\assets


INFO:tensorflow:Assets written to: weather_audio_detection_model\assets


In [ ]:
# Save model in h5 format
model.save('WeatherAudioDetectionModel.h5')

C:\Users\User\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
